In [1]:
from __future__ import print_function
import numpy as np
import os
import sys
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from libs.readfile_fau import *
import h5py

Using TensorFlow backend.


In [2]:
filename_train = './data/FAU_train.arff'
filename_test =  './data/FAU_test.arff'
train_data, train_label, train_size = readFile(filename_train)
test_data, test_label, test_size = readFile(filename_test)
#teacher_label=np.load("teacher_label.npy")
#train_label=teacher_label#使用teacher_label
fau_a,fau_e,fau_n,fau_p,fau_r=separa_fau(train_data,train_label)
train_data=normalize(train_data)
print("Anger:",fau_a.shape,"Emphatic:",fau_e.shape,"Neutral:",fau_n.shape,"Positive:",fau_p.shape,"Rest:",fau_r.shape)

Anger: (881, 256) Emphatic: (2093, 256) Neutral: (5590, 256) Positive: (674, 256) Rest: (721, 256)


In [3]:
def _to_tensor(x, dtype):
    x = tf.convert_to_tensor(x)
    if x.dtype != dtype:
        x = tf.cast(x, dtype)
    return x

def weighted_loss(r):
    
    def new_loss(y_true, y_pred):
        _EPSILON = 10e-8
        y_pred /= tf.reduce_sum(y_pred, reduction_indices=len(y_pred.get_shape()) - 1, keep_dims=True)
        # manual computation of crossentropy
        epsilon = _to_tensor(_EPSILON, y_pred.dtype.base_dtype)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        
        #y_true = tf.cast(y_true, tf.float64)
        rr = tf.cast(r, tf.float32)
        new_R = tf.matmul(rr, y_true, transpose_b=True)
        
        return - new_R * tf.reduce_sum(y_true * tf.log(y_pred), reduction_indices=len(y_pred.get_shape()) - 1)
        #return r * K.categorical_crossentropy(y_true, y_pred)

    return new_loss

In [4]:
weight_data=[]
weight_data.append([1.1,0.5,0.2,1.5,1.4])
#weight_data.append([1.0,1.0,1.0,1.0,1.0])
weight_data=np.asarray(weight_data)


model = Sequential()
model.add(Dense(30, activation='sigmoid', input_shape=(256,)))
#model.add(Dropout(0.5))
#model.add(Dense(15, activation='sigmoid'))
model.add(Dense(5, activation='softmax'))

#model.load_weights("./model/model_use.h5")#載入原網路

model.summary()
print("訓練資料：",train_data.shape,train_label.shape,train_size)
print("測試資料：",test_data.shape,test_label.shape)
sgd = SGD(lr=0.4, momentum=0.5, decay=0.0005)
model.compile(loss=weighted_loss(r=weight_data), optimizer=sgd)
history = model.fit(train_data, train_label,batch_size=100, epochs=600,verbose=2)
model.save_weights("./model/model.h5")
result=model.predict(test_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30)                7710      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 155       
Total params: 7,865.0
Trainable params: 7,865
Non-trainable params: 0.0
_________________________________________________________________
訓練資料： (9959, 256) (9959, 5) [881, 2093, 5590, 674, 721]
測試資料： (8257, 256) (8257, 5)
Epoch 1/600
0s - loss: 0.8253
Epoch 2/600
0s - loss: 0.7894
Epoch 3/600
0s - loss: 0.7608
Epoch 4/600
0s - loss: 0.7356
Epoch 5/600
0s - loss: 0.7229
Epoch 6/600
0s - loss: 0.7053
Epoch 7/600
0s - loss: 0.6976
Epoch 8/600
0s - loss: 0.6923
Epoch 9/600
0s - loss: 0.6881
Epoch 10/600
0s - loss: 0.6841
Epoch 11/600
0s - loss: 0.6773
Epoch 12/600
0s - loss: 0.6759
Epoch 13/600
0s - loss: 0.6748
Epoch 14/600
0s - loss: 0.6756
Epoch 15/600
0s - loss:

In [8]:
AC_A = 0.0
AC_E = 0.0
AC_N = 0.0
AC_P = 0.0
AC_R = 0.0
AC_WA = 0.0
AC_UA = 0.0
right=0#正確句數
matrix = np.zeros((5, 5))

use_data=result

for i, (Target, Label) in enumerate( zip(test_label,use_data) ) :  ### target:答案 label:辨識結果
    m = np.max(Label)#辨識結果（最大類別）
    for j, value in enumerate(Label) :  ### 分到哪類
        if value == m :
            for k, num in enumerate(Target) :  ### 正確為哪類
                if num == 1 :
                    matrix[k][j] += 1
                    break  # end of for k
            break  # end of for j

AC_A = matrix[0][0] / test_size[0]
AC_E = matrix[1][1] / test_size[1]
AC_N = matrix[2][2] / test_size[2]
AC_P = matrix[3][3] / test_size[3]
AC_R = matrix[4][4] / test_size[4]
print("\n-------------實驗結果-------------")
print("各類句數")
print("A:",test_size[0],"E:",test_size[1],"N:",test_size[2],"P:",test_size[3],"R:",test_size[4])
print("總句數:",len(use_data))
AC_UA = (AC_A + AC_E + AC_N + AC_P + AC_R) / 5
AC_WA = (matrix[0][0] + matrix[1][1] + matrix[2][2] + matrix[3][3] + matrix[4][4]) / len(use_data)
right = matrix[0][0] + matrix[1][1] + matrix[2][2] + matrix[3][3] + matrix[4][4]
print('      A      E      N      P      R')
print('A  %4d   %4d   %4d   %4d   %4d    ' % (matrix[0][0], matrix[0][1], matrix[0][2], matrix[0][3], matrix[0][4]))
print('E  %4d   %4d   %4d   %4d   %4d    ' % (matrix[1][0], matrix[1][1], matrix[1][2], matrix[1][3], matrix[1][4]))
print('N  %4d   %4d   %4d   %4d   %4d    ' % (matrix[2][0], matrix[2][1], matrix[2][2], matrix[2][3], matrix[2][4]))
print('P  %4d   %4d   %4d   %4d   %4d    ' % (matrix[3][0], matrix[3][1], matrix[3][2], matrix[3][3], matrix[3][4]))
print('R  %4d   %4d   %4d   %4d   %4d    ' % (matrix[4][0], matrix[4][1], matrix[4][2], matrix[4][3], matrix[4][4]))
print('\nA: %f    E: %f     N: %f     P: %f     R: %f\n' % (AC_A*100, AC_E*100, AC_N*100, AC_P*100, AC_R*100))
print("正確句數：",right)
print("正確率:",AC_UA*100)
print("=====================================================")


-------------實驗結果-------------
各類句數
A: 611 E: 1508 N: 5377 P: 215 R: 546
總句數: 8257
      A      E      N      P      R
A   389    144     59      1     18    
E   388    764    257      3     96    
N  1287   1284   2215     90    501    
P    30     17     78     35     55    
R   137     79    194     22    114    

A: 63.666121    E: 50.663130     N: 41.193974     P: 16.279070     R: 20.879121

正確句數： 3517.0
正確率: 38.5362832136


In [12]:
np.save("teacher_label",result)

In [13]:
teacher_label=np.load("teacher_label.npy")

In [14]:
teacher_label.shape

(8257, 5)

In [15]:
result[0,:]

array([ 0.18688405,  0.22766155,  0.01714887,  0.28681707,  0.28148845], dtype=float32)

In [6]:
print(train_data)

[[ 0.01112001  0.03305643  0.01097823 ...,  0.51146271  0.12504268
   0.04860263]
 [ 0.02209648  0.02864946  0.02205075 ...,  0.42486733  0.05899903
   0.08754948]
 [ 0.04212532  0.02661541  0.042124   ...,  0.44248585  0.07178136  0.07674   ]
 ..., 
 [ 0.36939974  0.02112745  0.36951982 ...,  0.4453776   0.07327221
   0.08830891]
 [ 0.06404232  0.01908711  0.06420513 ...,  0.31933371  0.01818332
   0.25570868]
 [ 0.04029312  0.01955332  0.04044564 ...,  0.28885606  0.01267977
   0.26501079]]


In [7]:
test_data = normalize(test_data)
result=model.predict(test_data)